Proposal:  State as a group node
==

This implementation follows the proposal to make State a node that can hold other nodes.

At the end of the notebook there is script, followed by the node calls it would result in.

In [16]:
class NodeGroup:
    def __init__(self):
        self.nodes = []
        self.context = None
        
    def added_to_scenegraph(self, graph):
        """
        Called when a node has been added to the scene graph.
        """
        pass

    def add_node(self, node):
        """
        Add a node to the group.
        
        The group is returned as the current context.
        """
        self.nodes.append(node)
        node.added_to_scenegraph(self)
        return self
    
    def push_group_node(self, node):
        """
        Push a group node to this group.
        
        The node itself is returned as the context.
        """
        self.nodes.append(node)
        node.context = node
        return node
    
    def __iter__(self):
        """
        Iterator that yields every node in the graph
        """
        for node in self.nodes:
            if isinstance(node, NodeGroup):
                yield from node
            else:
                yield node
        
class StateNode(NodeGroup):
    def __init__(self, **state):
        NodeGroup.__init__(self)
        self.state = state
    
    def added_to_scenegraph(self, graph):
        """
        Merge current state from the graph with the state here.
        """
        context = graph.context
        if context:
            self.state = dict(context.state, **self.state)
        graph.context = self
    
    def __repr__(self):
        return f'<StateNode {self.state}\n{self.nodes}>'

class PathNode:
    def __init__(self, coordinates):
        self.coordinates = coordinates
        
    def added_to_scenegraph(self, graph):
        pass
    
    def __repr__(self):
        return f'<PathNode {self.coordinates}>'
        
class SceneGraph(NodeGroup):
    def __init__(self):
        NodeGroup.__init__(self)
        
    def __repr__(self):
        return f'<SceneGraph {self.nodes}>'
        
class ConsoleRenderer:
    def render(self, graph):
        print(graph)
            
initial_state = {
    "fill": (1, 1, 1),
    "stroke": (1, 1, 1),
}

# background(1, 1, 1)
# with(fill=1, 0, 0)):
#     stroke(1, 1, 0)
#     rect(0, 0, 10, 10)

graph = SceneGraph()
context = graph \
    .push_group_node(StateNode(background=(1, 1, 1))) \
        .push_group_node(StateNode(fill=(1, 0, 0))) \
            .add_node(StateNode(stroke=(1, 0, 0))) \
            .add_node(PathNode(coordinates=[(0, 0), (10, 0), (10, 10), (0, 10)]))

renderer = ConsoleRenderer()
renderer.render(graph)

<SceneGraph [<StateNode {'background': (1, 1, 1)}
[<StateNode {'fill': (1, 0, 0)}
[<StateNode {'fill': (1, 0, 0), 'stroke': (1, 0, 0)}
[]>, <PathNode [(0, 0), (10, 0), (10, 10), (0, 10)]>]>]>]>


**Note**:  

Some open questions / slightly knarly details in this first cut:

I've distinguished between "NodeGroup" and SceneGraph, but maybe we could just have "SceneGraph".

This would make State as SceneGraph itself.

I thought it might be good to keep the concepts separate, but not so sure.